# Prerequisites

### Create User `dwhadmin`
Assumes user `dwhadmin` has privileges
- AdministratorAccess
- AmazonRedshiftFullAccess
- AmazonS3ReadOnlyAccess
- AmazonSSMFullAcces

In [ ]:
KEY ='YOUR_KEY_HERE'
SECRET='YOUR_SECRET_KEY_HERE'

### Create Role `dwhRole`

Assumes role `dwhRole` has privilege:
- AmazonS3ReadOnlyAccess

### Copy `dwhRole` ARN
Copy the role ARN into `dwh.cfg` 

(e.g. 'arn:aws:iam::123456789:role/dwhRole')

### Copy Cluster Host String

Finally, after starting the cluster from the console for the first time, copy the cluster hostname into the HOST field of `dwh.cfg` 

(e.g. 'dwhcluster.chbeihbcpsxt.us-east-1.redshift.amazonaws.com')

In [ ]:
import configparser
import pandas as pd
import boto3

config = configparser.ConfigParser()
config.read('dwh.cfg')

s3 = boto3.resource('s3',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                   )

redshift = boto3.client('redshift',
                        region_name="us-east-1",
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                       )

# Create Cluster

In [ ]:
try:
    response = redshift.create_cluster(        
        ClusterType=config['CLUSTER']['CLUSTER_TYPE'],
        NodeType=config['CLUSTER']['NODE_TYPE'],
        NumberOfNodes=int(config['CLUSTER']['NUM_NODES']),
        DBName=config['CLUSTER']['DB_NAME'],
        ClusterIdentifier=config['CLUSTER']['CLUSTER_IDENTIFIER'],
        MasterUsername=config['CLUSTER']['DB_USER'],
        MasterUserPassword=config['CLUSTER']['DB_PASSWORD'],
        IamRoles=[config['IAM_ROLE']['ARN']]  
    )
except Exception as e:
    print(e)

Wait a few minutes for the cluster to be created

# Create Tables

In [ ]:
!python create_tables.py

# Run ETL

In [ ]:
!python etl.py

# Test Queries

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(config['CLUSTER']['DB_USER'], 
                                                 config['CLUSTER']['DB_PASSWORD'], 
                                                 config['CLUSTER']['HOST'],
                                                 config['CLUSTER']['DB_PORT'],
                                                 config['CLUSTER']['DB_NAME'])
print(conn_string)

In [ ]:
%sql $conn_string

In [ ]:
%%sql
SELECT COUNT(*)
FROM artists;

In [ ]:
%%sql
SELECT COUNT(*)
FROM users;

In [ ]:
%%sql
SELECT COUNT(*)
FROM songs;

# Delete Cluster

In [ ]:
try:
    redshift.delete_cluster(
        ClusterIdentifier=config['CLUSTER']['CLUSTER_IDENTIFIER'],
        SkipFinalClusterSnapshot=True)
except Exception as e:
    print(e)